In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../")

import torch
import transformers
import baukit
from tqdm.auto import tqdm
import json
import os
from src import functional
import numpy as np
import logging
from src import models

from src.utils import logging_utils
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

torch.__version__, transformers.__version__, torch.version.cuda

/home/local_arnab/miniconda3/envs/sae/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


('2.4.1', '4.44.2', '12.1')

In [3]:
from datasets import load_dataset

wiki = load_dataset("wikimedia/wikipedia", "20231101.en")

2024-10-16 11:10:30 numexpr.utils INFO     Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-10-16 11:10:30 numexpr.utils INFO     NumExpr defaulting to 8 threads.
2024-10-16 11:10:30 datasets INFO     PyTorch version 2.4.1 available.
2024-10-16 11:10:30 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2024-10-16 11:10:30 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/wikimedia/wikipedia HTTP/11" 200 164229
2024-10-16 11:10:30 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): s3.amazonaws.com:443
2024-10-16 11:10:30 urllib3.connectionpool DEBUG    https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/wikimedia/wikipedia/wikimedia/wikipedia.py HTTP/11" 404 0
2024-10-16 11:10:30 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/wikimedia/wikipedia HTTP/11" 200 164229
2024-10-16 11:10:30 urllib3.connect

In [4]:
# print(len(wiki["train"]))
# wiki["train"][:5]

In [5]:
# tiny = load_dataset("roneneldan/TinyStories")

In [6]:
# print(len(tiny["train"]))
# tiny["train"][5055]

In [7]:
from src.models import ModelandTokenizer
from nnsight import LanguageModel

# model_name = "openai-community/gpt2-xl"
# model_name = "openai-community/gpt2"
# model_name = "EleutherAI/pythia-410m"
# model_name = "google/gemma-2-2b"
# model_name = "meta-llama/Llama-3.2-1B"
model_name = "Qwen/Qwen2.5-1.5B"

mt = ModelandTokenizer(
    model_key=model_name,
    # torch_dtype=torch.float16,
    torch_dtype=torch.float32,
)

# lm = LanguageModel(
#     model_key=model_name,
#     # torch_dtype=torch.float16,
#     device_map = "auto",
#     dispatch=True
# )
# lm.device

2024-10-16 11:10:34 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
2024-10-16 11:10:36 src.models INFO     loaded model </home/local_arnab/Codes/00_MODEL/Qwen/Qwen2.5-1.5B> | size: 9472.810 MB | dtype: torch.float32 | device: cuda:0


In [8]:
# # layer_name_format = 'gpt_neox.layers.{}'
# layer_name_format = mt.layer_name_format
# n_embd = mt.n_layer

In [9]:
# mt.embedder_name

In [10]:
# from src.functional import get_module_nnsight
# from src.functional import prepare_input

# prompts = [
#     "A quick brown fox jumps over the lazy",
#     "The capital of France is",
# ]    
# tokens = prepare_input(prompts, tokenizer=mt.tokenizer)

# tokens

In [11]:
# mt.tokenizer.padding_side

In [12]:
# test_tokens = prepare_input(
#     prompts, tokenizer=mt.tokenizer,
#     # truncation=True, max_length=5,
#     padding_side="right",
# )

# test_tokens

In [13]:
# test_tokens = prepare_input(
#     prompts, tokenizer=mt.tokenizer,
#     truncation=True, max_length=5,
#     # padding_side="left",
# )

# test_tokens

In [14]:
# with torch.inference_mode():
#     with mt.trace(test_tokens) as tr:
#         resid_out = get_module_nnsight(mt, layer_name_format.format(3)).output[0].save()
#         # input = get_module_nnsight(mt, mt.embedder_name).input.save()
#         # resid_in = get_module_nnsight(mt, layer_name_format.format(3)).input.save()
#         # output = mt.output.save()

# print(resid_out.shape)
# # resid_out

In [15]:
# with torch.inference_mode():
#     with mt.trace(
#         prompts,
#         invoker_args={"truncation": True, "max_length": 5}
#     ) as tr:
#         resid_out_2 = get_module_nnsight(mt, layer_name_format.format(3)).output[0].save()
#         # input = get_module_nnsight(mt, mt.embedder_name).input.save()
#         # resid_in = get_module_nnsight(mt, layer_name_format.format(3)).input.save()
#         # output = mt.output.save()

# print(resid_out_2.shape)
# # resid_out_2

In [16]:
# torch.allclose(resid_out, resid_out_2)

In [17]:
mt.n_embd

1536

In [24]:
from dictionary_learning import ActivationBuffer
from dictionary_learning.training import trainSAE
from src.functional import get_module_nnsight

submodule = get_module_nnsight(mt, mt.layer_name_format.format(6))
activation_dim = mt.n_embd
dictionary_dim = 16384

# data_iter = iter(tiny["train"][:2100000]["text"])
# data_iter = iter(tiny["train"][:10000]["text"])
data_iter = iter(wiki["train"][:1200]["text"])


tiny_buffer = ActivationBuffer(
    data_iter,
    mt,
    submodule,
    d_submodule=activation_dim, 
    n_ctxs=26, 
    ctx_len=256,
    
    refresh_batch_size=26,
    device='cuda:0', 
)

tiny_buffer.config

init self.activations.shape=torch.Size([0, 1536])
init self.read.shape=torch.Size([0])


{'d_submodule': 1536,
 'io': 'out',
 'n_ctxs': 26,
 'ctx_len': 256,
 'refresh_batch_size': 26,
 'out_batch_size': 8192,
 'device': 'cuda:0'}

In [25]:
counter = 0
while True:
    try:
        tiny_buffer.text_batch()
        counter += 1
        print(f"{counter=}")
    except StopIteration:
        break
counter

counter=1
counter=2
counter=3
counter=4
counter=5
counter=6
counter=7
counter=8
counter=9
counter=10
counter=11
counter=12
counter=13
counter=14
counter=15
counter=16
counter=17
counter=18
counter=19
counter=20
counter=21
counter=22
counter=23
counter=24
counter=25
counter=26
counter=27
counter=28
counter=29
counter=30
counter=31
counter=32
counter=33
counter=34
counter=35
counter=36
counter=37
counter=38
counter=39
counter=40
counter=41
counter=42
counter=43
counter=44
counter=45
counter=46


46

In [21]:
from src.functional import free_gpu_cache

free_gpu_cache()

In [22]:
! rm -rf trained_saes
! rm -rf wandb

In [23]:
from dictionary_learning.trainers import TrainerTopK, GatedSAETrainer, GatedAnnealTrainer, StandardTrainer
from dictionary_learning.dictionary import AutoEncoder
from src.utils.experiment_utils import set_seed

set_seed(1234)

# train the sparse autoencoder (SAE)
ae = trainSAE(
    data = tiny_buffer,
    trainer_configs= [{
        'trainer' : GatedSAETrainer,
        # 'dict_class' : AutoEncoder,
        'activation_dim' : activation_dim,
        'dict_size' : dictionary_dim,
        'lr' : 1e-3,
        # 'l1_penalty' : 1e-2,
        'warmup_steps' : 3,
        'resample_steps' : None,
        'seed' : None,
        'wandb_name' : 'GatedSAETrainer',
        'lm_name': mt.name,
        'layer': submodule,
        'submodule_name': "residual",
    }],
    save_dir="trained_saes",
    save_steps=10,

    # use_wandb=True,
    # wandb_entity="dl-homeworks",
    # wandb_project="test_sae",
    # log_steps=5
)

2024-10-16 10:31:03 src.utils.experiment_utils INFO     setting all seeds to 1234


['activation_dim', 'dict_size', 'lr', 'warmup_steps', 'resample_steps', 'seed', 'wandb_name', 'lm_name', 'layer', 'submodule_name']


0it [00:00, ?it/s]You're using a Qwen2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
28it [01:12,  2.59s/it]


In [26]:
from dictionary_learning.dictionary import AutoEncoder, GatedAutoEncoder
from dictionary_learning.trainers.top_k import AutoEncoderTopK
from src.utils import env_utils

ae_0 = GatedAutoEncoder.from_pretrained(
    path = os.path.join(
        env_utils.DEFAULT_RESULTS_DIR,
        "train_sae",
        "Qwen2.5-1.5B",
        "wikipedia",
        str(10000),
        "trainer_0/checkpoints/ae_0.pt"
    ),
    # k = 100,
    device='cuda:0'
)

/home/local_arnab/Codes/Projects/sae/notebooks/../dictionary_learning/dictionary.py:206: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = t.load(path)


In [27]:
ae_0

GatedAutoEncoder(
  (encoder): Linear(in_features=1536, out_features=16384, bias=False)
  (decoder): Linear(in_features=16384, out_features=1536, bias=False)
)

In [28]:
ae_0.encoder.weight

Parameter containing:
tensor([[ 0.0002,  0.0162, -0.0010,  ..., -0.0243, -0.0069, -0.0150],
        [-0.0141,  0.0139,  0.0003,  ..., -0.0059, -0.0178,  0.0090],
        [-0.0002, -0.0115, -0.0247,  ...,  0.0135,  0.0073,  0.0149],
        ...,
        [ 0.0206, -0.0046,  0.0235,  ...,  0.0177,  0.0241, -0.0002],
        [-0.0016,  0.0002,  0.0163,  ...,  0.0073, -0.0092, -0.0223],
        [-0.0065, -0.0244, -0.0111,  ...,  0.0114, -0.0159, -0.0097]],
       device='cuda:0', requires_grad=True)

In [29]:
ae_1 = GatedAutoEncoder.from_pretrained(
    path = os.path.join(
        env_utils.DEFAULT_RESULTS_DIR,
        "train_sae",
        "Qwen2.5-1.5B",
        "wikipedia",
        str(10000),
        "trainer_0/checkpoints/ae_60.pt"
    ),
    # k = 100,
    device='cuda:0'
)

In [30]:
ae_1.encoder.weight

Parameter containing:
tensor([[-1.9945e-04,  1.6562e-02, -6.1725e-04,  ..., -2.3844e-02,
         -7.3131e-03, -1.4618e-02],
        [-1.4073e-02,  1.3654e-02, -2.5202e-05,  ..., -5.1585e-03,
         -1.7493e-02,  9.6451e-03],
        [-6.2147e-04, -1.1176e-02, -2.4371e-02,  ...,  1.3880e-02,
          6.9265e-03,  1.5251e-02],
        ...,
        [ 2.0845e-02, -4.1893e-03,  2.3065e-02,  ...,  1.8006e-02,
          2.3994e-02, -6.1753e-05],
        [-1.3185e-03, -8.2879e-05,  1.5939e-02,  ...,  6.9403e-03,
         -8.9061e-03, -2.2619e-02],
        [-6.1018e-03, -2.4697e-02, -1.1541e-02,  ...,  1.0957e-02,
         -1.5486e-02, -1.0146e-02]], device='cuda:0', requires_grad=True)

In [31]:
torch.allclose(ae_0.encoder.weight, ae_1.encoder.weight, atol=1e-3)

False